In [7]:
# Import libraries.
using Turing, StatsPlots, Random

┌ Info: Precompiling StatsPlots [f3b207a7-027a-5e70-b257-86293d7955fd]
└ @ Base loading.jl:1278


In [23]:
# Declare our Turing model.
@model function rps_agent(action)
    # Our prior belief about the probability of RPS.
    alpha ~ Dirichlet(ones(3)/3)
    action ~ Categorical(vec(alpha))
    return action
end

rps_agent (generic function with 2 methods)

In [24]:
# Settings of the sampler.
iterations = 1000

# Start sampling.
chain = sample(rps_agent(1.0), PG(10), iterations);

Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00


In [25]:
chain

Chains MCMC chain (1000×5×1 Array{Float64,3}):

Log evidence      = -1.1568848367219688
Iterations        = 1:1000
Thinning interval = 1
Chains            = 1
Samples per chain = 1000
parameters        = alpha[1], alpha[2], alpha[3]
internals         = logevidence, lp

Summary Statistics
  parameters      mean       std   naive_se      mcse         ess      rhat 
      Symbol   Float64   Float64    Float64   Float64     Float64   Float64 

    alpha[1]    0.6392    0.2827     0.0089    0.0089    955.7900    0.9995
    alpha[2]    0.1841    0.2293     0.0073    0.0094   1072.2105    0.9991
    alpha[3]    0.1768    0.2245     0.0071    0.0085   1032.3805    0.9994

Quantiles
  parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
      Symbol   Float64   Float64   Float64   Float64   Float64 

    alpha[1]    0.0687    0.4259    0.6959    0.8894    0.9970
    alpha[2]    0.0000    0.0100    0.0733    0.2947    0.7715
    alpha[3]    0.0000    0.0085    0.0774    0.2635    0.8083

In [36]:
# Declare our Turing model.
@model function rps_agent2(agent1, action)
    # Our prior belief about the probability of RPS.
    alpha ~ Dirichlet(ones(3)/3)
    action ~ Categorical(vec(alpha))
    opp_action = sample(agent1(action), PG(10), 10);
    return action
end

rps_agent2 (generic function with 1 method)

In [37]:
rps_agent2(rps_agent, 1)

DynamicPPL.Model{var"#19#20",(:agent1, :action),(),(),Tuple{typeof(rps_agent),Int64},Tuple{}}(:rps_agent2, var"#19#20"(), (agent1 = rps_agent, action = 1), NamedTuple())

In [38]:
chain = sample(rps_agent2(rps_agent, 1.0), PG(10), 10);

Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00


In [39]:
chain

Chains MCMC chain (10×5×1 Array{Float64,3}):

Log evidence      = -1.0383018851051466
Iterations        = 1:10
Thinning interval = 1
Chains            = 1
Samples per chain = 10
parameters        = alpha[1], alpha[2], alpha[3]
internals         = logevidence, lp

Summary Statistics
  parameters      mean       std   naive_se      mcse         ess      rhat 
      Symbol   Float64   Float64    Float64   Missing     Float64   Float64 

    alpha[1]    0.4696    0.3413     0.1079   missing    -69.1831    0.9359
    alpha[2]    0.2689    0.2957     0.0935   missing      4.5290    1.1888
    alpha[3]    0.2614    0.3383     0.1070   missing   -128.7490    0.9500

Quantiles
  parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
      Symbol   Float64   Float64   Float64   Float64   Float64 

    alpha[1]    0.0383    0.1803    0.4361    0.7600    0.9361
    alpha[2]    0.0090    0.0567    0.1294    0.4894    0.7772
    alpha[3]    0.0012    0.0070    0.0952    0.4666    0.8577


In [46]:
# Declare our Turing model.
@model function agent(other_agent, action, other_agents_depth=0, depth=0, discrete_sampler = PG, discrete_sampler_hyper_param=10, num_of_iterations=10)
    # Our prior belief about the probability of RPS.
    alpha ~ Dirichlet(ones(3)/3)
    if depth > 0 
        opp_action = sample(other_agent(agent, action, depth-1, other_agents_depth), discrete_sampler(discrete_sampler_hyper_param), num_of_iterations);
    end
    action ~ Categorical(vec(alpha))
    return action
end

agent (generic function with 6 methods)

In [47]:
chain = sample(agent(agent, 1.0), PG(10), 10);

Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00


In [48]:
chain

Chains MCMC chain (10×5×1 Array{Float64,3}):

Log evidence      = -1.1091171317987318
Iterations        = 1:10
Thinning interval = 1
Chains            = 1
Samples per chain = 10
parameters        = alpha[1], alpha[2], alpha[3]
internals         = logevidence, lp

Summary Statistics
  parameters      mean       std   naive_se      mcse        ess      rhat 
      Symbol   Float64   Float64    Float64   Missing    Float64   Float64 

    alpha[1]    0.7423    0.2406     0.0761   missing    27.5690    0.9040
    alpha[2]    0.0756    0.0771     0.0244   missing   -22.8194    0.8966
    alpha[3]    0.1821    0.2228     0.0705   missing    14.4741    0.9094

Quantiles
  parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
      Symbol   Float64   Float64   Float64   Float64   Float64 

    alpha[1]    0.3494    0.6076    0.7850    0.9410    0.9951
    alpha[2]    0.0020    0.0245    0.0566    0.0865    0.2259
    alpha[3]    0.0002    0.0075    0.0712    0.3289    0.5843


In [72]:
chain = sample(agent(agent, 1.0, 1, 1, PG, 10, 10), PG(1), 10);

Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:06


In [73]:
chain

Chains MCMC chain (10×5×1 Array{Float64,3}):

Log evidence      = -2.192487260729494
Iterations        = 1:10
Thinning interval = 1
Chains            = 1
Samples per chain = 10
parameters        = alpha[1], alpha[2], alpha[3]
internals         = logevidence, lp

Summary Statistics
  parameters      mean       std   naive_se      mcse        ess      rhat 
      Symbol   Float64   Float64    Float64   Missing    Float64   Float64 

    alpha[1]    0.3184    0.3164     0.1001   missing   -34.5372    0.9134
    alpha[2]    0.3656    0.3608     0.1141   missing     9.6475    1.0854
    alpha[3]    0.3159    0.3111     0.0984   missing     7.1931    1.0188

Quantiles
  parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
      Symbol   Float64   Float64   Float64   Float64   Float64 

    alpha[1]    0.0019    0.0905    0.2417    0.4061    0.9000
    alpha[2]    0.0108    0.0432    0.2532    0.6832    0.9338
    alpha[3]    0.0011    0.0296    0.2469    0.5647    0.7976


In [74]:
mean(chain[:"alpha[1]"])

2-dimensional AxisArray{Float64,2,...} with axes:
    :iter, 1:1:10
    :chain, 1:1
And data, a 10×1 Array{Float64,2}:
 0.0009246402037742542
 0.23180987127175853
 0.3779121639059845
 0.7589982256345258
 0.005106611984183574
 0.2515607406398114
 0.9409139622121607
 0.08025311206660969
 0.4155011826260939
 0.12135360869896437